# Demo of loading options

In [ ]:
import datetime as dt
import pandas as pds

In [ ]:
import pysat

# Set data directory if user hasn't already set one
if len(pysat.params['data_dirs']) == 0:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = '~/shared/pysatDemo'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

In [ ]:
# All of the examples work with any of the Instrument types below.

# Basic test instrument, pandas format
inst = pysat.Instrument('pysat', 'testing')

# Basic test instrument, xarray format
# inst = pysat.Instrument('pysat', 'testing_xarray')

# Test instrument, higher dimensional data, xarray format
# inst = pysat.Instrument('pysat', 'testing2d_xarray')

# Basic test model, xarray format
# inst = pysat.Instrument('pysat', 'testmodel', tag='pressure_levels')

# Load by year, day of year

In [ ]:
# Load by year, doy
inst.load(2009, 1, use_header=True)

In [ ]:
# Load a range of data. Inclusive first input, exclusive second.
# Keywords starting with `end_` are exclusing.
# This load is equivalent to one above.
inst.load(2009, 1, end_yr=2009, end_doy=2, use_header=True)
inst.data

In [ ]:
# Load a range of data. Inclusive first input, exclusive second.
# Keywords starting with `end_` are exclusing.
# This loads two days of data.
inst.load(2009, 1, end_yr=2009, end_doy=3, use_header=True)
inst.data

# Load by datetime(s)

In [ ]:
# Load by date
inst.load(date=dt.datetime(2009, 1, 1), use_header=True)
inst.data

In [ ]:
# Load by range of dates
inst.load(date=dt.datetime(2009, 1, 1), end_date=dt.datetime(2009, 1, 3), use_header=True)
inst.data

# Load by filename(s)

In [ ]:
inst.load(fname=inst.files.files[0], use_header=True)
inst.data

In [ ]:
# Filename bounds are inclusive for both ends
# Keywords ending with `stop_` are inclusive.
inst.load(fname=inst.files.files[0], stop_fname=inst.files.files[1], use_header=True)
inst.data

# Load All Data

In [ ]:
# Create new Instrument test object with limited file range to demonstrate loading all data
inst = pysat.Instrument('pysat', 'testing', file_date_range=[dt.datetime(2019, 1, 1), dt.datetime(2019, 1, 3)])

# Load all simulated data
inst.load(use_header=True)

# Print to Jupyter
inst.data

# Custom instrument module keywords

In [ ]:
# Keywords may be passed to underlying instrument support modules via pysat. Keywords passed like
# this only apply for a single function call.
inst.load(2009, 1, num_samples=10, use_header=True)
inst.data

In [ ]:
# Keywords may also be supplied at instantiation. These values persist.
inst = pysat.Instrument('pysat', 'testing', num_samples=10)
inst.load(2009, 1, use_header=True)
inst.data

In [ ]:
# `num_samples` may be modified on subsequent load calls, but, unless otherwise specified, `num_samples`
# will return to the value at instantiation.
inst.load(2009, 1, use_header=True, num_samples=3)
inst.data

In [ ]:
# Confirmation that `num_samples` consistent with instantiation value
inst.load(2009, 1, use_header=True)
inst.data

In [ ]:
# Note the `Data Processing` section below. pyast passes supplied keywords to the appropriate methods.
print(inst)

In [ ]:
# All keywords in the instrument support module identified by pysat are stored as `kwargs_supported` attribute
for key in inst.kwargs_supported:
    print('Keywords for Instrument method: ', key)
    print('\t', repr(inst.kwargs_supported[key]), '\n')

# Data Padding for Time-Based Calculations

In [ ]:
# Some calculations in time require a 'spin-up'/'spin-down' for full accuracy. For example, a standard
# deviation over 30 seconds requires 30 seconds of data. For a centered standard deviation this is a 
# challenge for the first sample in a day. There isn't a preceding 15 seconds of data. To make accurate 
# time based calculations easy, pysat can load buffer data from the next/prev today, 
# join it with the current days data, then run the custom functions,
# and finally, pysat removes the buffer data, leaving just the desired time range.
# This happens in the background and requires no explicit support by any instrument module developer.

# Define data padding window
pad = pds.DateOffset(seconds=30)

# Instantiate instrument with padding information. 
inst = pysat.Instrument('pysat', 'testing', pad=pad)

# Load data. The `verifyPad` keyword, if True, retains
# the padded data. Default is False.
inst.load(2009, 1, verifyPad=True, use_header=True)

# Use Jupyter to display data
inst.data